In [ ]:
%load_ext autoreload
%autoreload 2

# Exercise 1



<img src='./images/01.png' width=800>

In [ ]:
import os
import mlflow
os.environ['MLFLOW_TRACKING_URI'] = './mlruns03_1'
mlflow.set_tracking_uri(os.environ.get('MLFLOW_TRACKING_URI'))

In [ ]:
mlflow.set_experiment('Exercise_1')

2025/04/09 08:42:38 INFO mlflow.tracking.fluent: Experiment with name 'Exercise_1' does not exist. Creating a new experiment.


<Experiment: artifact_location='/kaggle/working/mlruns/674194671375701503', creation_time=1744188158264, experiment_id='674194671375701503', last_update_time=1744188158264, lifecycle_stage='active', name='Exercise_1', tags={}>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
import torch.optim as optim
from torchinfo import summary
from utils import train_network, accuracy_score_wrapper


In [ ]:
mnist_data_train = torchvision.datasets.MNIST(
    "./data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
    )
mnist_data_test = torchvision.datasets.MNIST(
    "./data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
    )

batch_size = 32
mnist_train_loader = DataLoader(
    mnist_data_train,
    batch_size=batch_size,
    shuffle=True
    )
mnist_test_loader = DataLoader(
    mnist_data_test,
    batch_size=batch_size
    )

In [ ]:
train_transforms = transforms.Compose(
    [
        transforms.RandomAffine(
            degrees=5,
            translate=(0.05, 0.05),
            scale=(0.98, 1.02),
            ),
        transforms.ToTensor(),
    ]
)
test_transforms = transforms.ToTensor()
mnist_train_augmented = torchvision.datasets.MNIST(
    './data_augmented',
    train=True,
    download=True,
    transform=train_transforms,
)
mnist_test_augmented = torchvision.datasets.MNIST(
    './data_augmented',
    train=False,
    download=True,
    transform=test_transforms,
)

mnist_train_augmented_loader = DataLoader(
    mnist_train_augmented,
    shuffle=True,
    batch_size=batch_size,
    num_workers=4
    )
mnist_test_augmented_loader = DataLoader(
    mnist_test_augmented,
    batch_size=batch_size,
    num_workers=4
    )

In [ ]:
#models
import torch.nn as nn


def model_linear(D, hidden_layer_neurons, classes):
    model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(D, hidden_layer_neurons),
    nn.Tanh(),
    nn.Linear(hidden_layer_neurons, classes),
    )
    return model

def model_cnn(D, C, filters, K, classes):
    model = nn.Sequential(
    nn.Conv2d(C, filters, K, padding=K//2),
    nn.Tanh(),
    nn.Flatten(),
    nn.Linear(filters*D, classes),
    )
    return model

def model_cnn_pool(D, C, filters, K, classes):
    model = nn.Sequential(
    nn.Conv2d(C, filters, K, padding=K//2),
    nn.Tanh(),
    nn.Conv2d(filters, filters, K, padding=K//2),
    nn.Tanh(),
    nn.Conv2d(filters, filters, K, padding=K//2),
    nn.Tanh(),
    nn.MaxPool2d(2),
    nn.Conv2d(filters, 2*filters, K, padding=K//2),
    nn.Tanh(),
    nn.Conv2d(2*filters, 2*filters, K, padding=K//2),
    nn.Tanh(),
    nn.Conv2d(2*filters, 2*filters, K, padding=K//2),
    nn.Tanh(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(2*filters*D//4**2, classes),
    )
    return model



In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
loss_func = nn.CrossEntropyLoss()
score_funcs = {"Accuracy": accuracy_score_wrapper}
epochs = 40
params = {
    'device': device,
    'loss_func': loss_func.__class__.__name__,
    'epochs': epochs,
    'batch_size': batch_size
}
def plot_results(data_df, close=True):
    sns.lineplot(data_df, x='epoch', y='valid Acc', label='valid Acc')
    plt.xlabel('epoch')
    plt.ylabel('valid Acc')
    plt.title('valid Acc')
    fig = plt.gcf()
    if close:
        plt.close()
    return fig

In [ ]:

models = (model_linear(D=28*28, hidden_layer_neurons=256, classes=10),
        model_cnn(D=28*28, C=1, filters=16, K=3, classes=10),
        model_cnn_pool(D=28*28, C=1, filters=16, K=3, classes=10),
        model_cnn_pool(D=28*28, C=1, filters=16, K=3, classes=10))
train_loaders = (mnist_train_loader, mnist_train_loader, mnist_train_loader, mnist_train_augmented_loader)
test_loaders = (mnist_test_loader, mnist_test_loader, mnist_test_loader, mnist_test_augmented_loader)
name_experiment = (
    'linear_mnist',
    'cnn_mnist',
    'cnn_pool_mnist',
    'cnn_pool_augmented_mnist'
)

In [ ]:
for i in range(4):
    print(i)
    model = models[i]
    train_loader = train_loaders[i]
    test_loader = test_loaders[i]
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    params['optimizer'] = optimizer.defaults
    
    with open('model_summary.txt', 'w') as f:
        f.write(str(summary(model)))
    with mlflow.start_run(nested=True, run_name=name_experiment[i]):
        mlflow.log_params(params)
        mlflow.log_artifact('model_summary.txt')

        results = train_network(
            model=model,
            optimizer=optimizer,
            loss_func=loss_func,
            train_loader=train_loader,
            valid_loader=test_loader,
            epochs=epochs,
            device=device,
            score_funcs=score_funcs,
            )

<img src="./images/valid_acc_step10.png" width=1000>

<img src="./images/valid_acc_step40.png" width=1000>

<img src="./images/valid_loss_step10.png" width=1000>

<img src="./images/valid_loss_step40.png" width=1000>